In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-191147
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-191147


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpu-cluster"
cluster_name = "Manual-Compute"

# HK-Step 3: Create computer Cluster 
# I used the code from the nyc-taxi code example 
# 
# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name) 
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)


Found existing cluster, use it.

Running


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# HK-Step 4: Specify parameter sampler
# See the Keras example code
#
ps = RandomParameterSampling(
{
    '--C' : choice(0.01,0.1,1,10,100),
    '--max_iter' : choice(20,40,70,100)
})


# HK-Step 5:  Specify a Policy
# See the Keras example code
#
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# HK-Step 6: Create a ScriptRunConfig Object to specify the configuration details of your training job
# See the Keras example code
#

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env)

# HK-Step 7:Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# See the Keras example code
#
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)


In [9]:
# HK-Step 7: Submit your hyperdrive run to the experiment and show run details with the widget.
# See the Keras example code
#
hyperdrive_run = exp.submit(config=hyperdrive_config)

# We can use a run history widget to show the progress
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

#hyperdrive_run.get_status()

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_79914e2f-00ae-4b36-884e-019c606195b5
Web View: https://ml.azure.com/runs/HD_79914e2f-00ae-4b36-884e-019c606195b5?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-191147/workspaces/quick-starts-ws-191147&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-04-06T08:22:10.244738][API][INFO]Experiment created<END>\n""<START>[2022-04-06T08:22:10.970772][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-04-06T08:22:11.583361][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_79914e2f-00ae-4b36-884e-019c606195b5
Web View: https://ml.azure.com/runs/HD_79914e2f-00ae-4b36-884e-019c606195b5?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-191147/workspaces/quick-starts-ws-191147&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



KeyError: 'log_files'

In [18]:
import joblib
# HK-Step 8: Get your best run and save the model from that run.
# See the Keras example code
# 

best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("best-run details :", best_run.get_details())
print('\n')
print("best-run id : ",best_run.id)
print('\n')
print("best-run file names :", best_run.get_file_names())
print('\n')
print("best-run metrics :", best_run.get_metrics())
best_run.upload_file(name="bestrunherman", path_or_stream = 'outputs/model')
model = best_run.register_model(model_name = 'bestrunherman', model_path='outputs/model')


best-run details : {'runId': 'HD_79914e2f-00ae-4b36-884e-019c606195b5_14', 'target': 'Manual-Compute', 'status': 'Completed', 'startTimeUtc': '2022-04-06T08:36:27.931987Z', 'endTimeUtc': '2022-04-06T08:37:19.61937Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '94c010b5-aa16-4f5c-9dbd-70fcec539b0c', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.1', '--max_iter', '70'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'Manual-Compute', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'skle

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/batch/tasks/shared/LS_root/mounts/clusters/manual-compute/code/Users/odl_user_191147/outputs/model'

In [19]:
from azureml.data.dataset_factory import TabularDatasetFactory

# HK-Step 9: Create TabularDataset using TabularDatasetFactory
# 

data_location = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=data_location)



In [20]:
from train import clean_data

# HK-Step 10: Use the clean_data function to clean your data.
#
x, y = clean_data(ds)

In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)


In [22]:
# Submit your automl run

remote_run = exp.submit(config=automl_config, show_output=True)
remote_run.wait_for_completion()


No run_configuration provided, running on local with default configuration
Running in the active local environment.


2022-04-06:09:22:30,566 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-04-06:09:22:30,573 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_8579fd6b-75bd-44d3-a56a-ad812c8bf852,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-04-06:09:51:41,980 INFO     [explanation_client.py:334] Using default datastore for uploads


{'runId': 'AutoML_8579fd6b-75bd-44d3-a56a-ad812c8bf852',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-04-06T09:22:45.145223Z',
 'endTimeUtc': '2022-04-06T09:49:43.525165Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-191147","workspace_name":"quick-starts-ws-191147","region":"southcentralus","compute_target":"local","spark_service":nul

In [26]:
# HK-Step 11: Retrieve and save your best automl model.

best_run_auto, fitted_model = remote_run.get_output()

print(best_run_auto)
print(fitted_model)

print("best-run details :", best_run_auto.get_details())
print('\n')
print("best-run id : ",best_run_auto.id)
print('\n')
print("best-run file names :", best_run_auto.get_file_names())
print('\n')
print("best-run metrics :", best_run_auto.get_metrics())

best_run_auto.register_model(model_name = 'best_automl_register', model_path = '/outputs/')

Run(Experiment: udacity-project,
Id: AutoML_8579fd6b-75bd-44d3-a56a-ad812c8bf852_30,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('11', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), rando

Model(workspace=Workspace.create(name='quick-starts-ws-191147', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-191147'), name=best_automl_register, id=best_automl_register:1, version=1, tags={}, properties={})